In [1]:
using IterativeLQR
using StaticArrays
using LinearAlgebra
using Plots
using QuadrotorDynamics
using Symbolics

┌ Info: Precompiling QuadrotorDynamics [f6d16423-25c2-4b6a-a4a4-258ba236e06a]
└ @ Base loading.jl:1342


In [2]:
function Dynamics(f::Function, nx::Int, nu::Int; nw::Int=0)
    #TODO: option to load/save methods
    @variables x[1:nx], u[1:nu], w[1:nw] 
    y = f(x, u, w) 
    jacx = Symbolics.jacobian(y, x);
    jacu = Symbolics.jacobian(y, u);
    val_func = eval(Symbolics.build_function(y, x, u, w)[2]);
    jacx_func = eval(Symbolics.build_function(jacx, x, u, w)[2]);
    jacu_func = eval(Symbolics.build_function(jacu, x, u, w)[2]);
    ny = length(y)

    return IterativeLQR.Dynamics(val_func, jacx_func, jacu_func, 
                    ny, nx, nu, nw, 
                    zeros(ny), zeros(ny, nx), zeros(ny, nu))
end

function Cost(f::Function, nx::Int, nu::Int; nw::Int=0)
    #TODO: option to load/save methods
    @variables x[1:nx], u[1:nu], w[1:nw]
    
    val = f(x, u, w)
    gradx = Symbolics.gradient(val, x)
    gradu = Symbolics.gradient(val, u) 
    hessxx = Symbolics.jacobian(gradx, x) 
    hessuu = Symbolics.jacobian(gradu, u) 
    hessux = Symbolics.jacobian(gradu, x) 

    val_func = eval(Symbolics.build_function([val], x, u, w)[2])
    gradx_func = eval(Symbolics.build_function(gradx, x, u, w)[2])
    gradu_func = eval(Symbolics.build_function(gradu, x, u, w)[2])
    hessxx_func = eval(Symbolics.build_function(hessxx, x, u, w)[2])
    hessuu_func = eval(Symbolics.build_function(hessuu, x, u, w)[2])
    hessux_func = eval(Symbolics.build_function(hessux, x, u, w)[2])  

    return IterativeLQR.Cost(val_func, 
        gradx_func, gradu_func, 
        hessxx_func, hessuu_func, hessux_func,
        zeros(1), 
        zeros(nx), zeros(nu), 
        zeros(nx, nx), zeros(nu, nu), zeros(nu, nx))
end


Cost (generic function with 1 method)

In [3]:
# ## horizon 
T = 101 
dt = 0.01

# ## quadrotor
nx = 13 
nu = 4
nw = 0 

# ## model
rotor = Rotor(2, 0.165, 0.018, 0.0, 0.005, 0.010, 0.004, 0.0048, 0.0048*sqrt(0.0024), 6.8*π/180, 14.6*π/180, 5.5)
quad = QuadRotor(4., SMatrix{3,3}(Diagonal([0.082, 0.082, 0.149])), -0.007, 0.315, 0.05, rotor, false)
dyn = Dynamics((x, u, w) -> QuadrotorDynamics.step(quad, x, u; h=dt), nx, nu; nw=nw)
model = [dyn for t = 1:T-1]

# ## initialization
x1 = [1; 0; 0; 1; zeros(3); zeros(3); zeros(3)]
xT = [zeros(3); 1; zeros(3); zeros(3); zeros(3)]
ū = [1.0 * randn(nu) for t = 1:T-1] 
w = [zeros(nw) for t = 1:T]
x̄ = rollout(model, x1, ū, w)

# ## objective 
# Bryson's rule: max position deviation ~5, max rad/sec ~1500, 
ot = (x, u, w) -> 1/(5^2) * (dot(x[1:3], x[1:3]) + dot(x[8:13], x[8:13])) + 1/(1500^2) * dot(u, u)
oT = (x, u, w) -> 0.1 * (dot((x-xT)[1:3], (x-xT)[1:3]) + dot((x-xT)[8:13], (x-xT)[8:13]))
ct = Cost(ot, nx, nu; nw=nw)
cT = Cost(oT, nx, 0; nw=nw)
obj = [[ct for t = 1:T-1]..., cT]

# ## constraints
goal(x, u, w) = x - xT

cont = Constraint()
conT = Constraint(goal, nx, 0)
cons = [[cont for t = 1:T-1]..., conT] 

# ## problem
prob = solver(model, obj, cons)
initialize_controls!(prob, ū)
initialize_states!(prob, x̄)

# ## solve
solve!(prob)

# ## solution
x_sol, u_sol = get_trajectory(prob)

LoadError: InterruptException:

In [ ]:
# ## horizon 
T = 101 

# ## acrobot 
nx = 4 
nu = 1 
nw = 0 

function acrobot(x, u, w)
    mass1 = 1.0  
    inertia1 = 0.33  
    length1 = 1.0 
    lengthcom1 = 0.5 

    mass2 = 1.0  
    inertia2 = 0.33  
    length2 = 1.0 
    lengthcom2 = 0.5 

    gravity = 9.81 
    friction1 = 0.1 
    friction2 = 0.1

    function M(x, w)
        a = (inertia1 + inertia2 + mass2 * length1 * length1
            + 2.0 * mass2 * length1 * lengthcom2 * cos(x[2]))

        b = inertia2 + mass2 * length1 * lengthcom2 * cos(x[2])

        c = inertia2

       return [a b; b c]
    end

    function τ(x, w)
        a = (-1.0 * mass1 * gravity * lengthcom1 * sin(x[1])
            - mass2 * gravity * (length1 * sin(x[1])
            + lengthcom2 * sin(x[1] + x[2])))

        b = -1.0 * mass2 * gravity * lengthcom2 * sin(x[1] + x[2])

        return [a; b]
    end

    function C(x, w)
        a = -2.0 * mass2 * length1 * lengthcom2 * sin(x[2]) * x[4]
        b = -1.0 * mass2 * length1 * lengthcom2 * sin(x[2]) * x[4]
        c = mass2 * length1 * lengthcom2 * sin(x[2]) * x[3]
        d = 0.0

        return [a b; c d]
    end

    function B(x, w)
        [0.0; 1.0]
    end

    q = view(x, 1:2)
    v = view(x, 3:4)

    qdd = M(q, w) \ (-1.0 * C(x, w) * v
            + τ(q, w) + B(q, w) * u[1] - [friction1; friction2] .* v)

    return [x[3]; x[4]; qdd[1]; qdd[2]]
end

function midpoint_explicit(x, u, w)
    h = 0.1 # timestep 
    x + h * acrobot(x + 0.5 * h * acrobot(x, u, w), u, w)
end

# ## model
dyn = Dynamics(midpoint_explicit, nx, nu; nw=nw)
model = [dyn for t = 1:T-1] 

# ## initialization
x1 = [0.0; 0.0; 0.0; 0.0] 
xT = [0.0; π; 0.0; 0.0]
ū = [1.0 * randn(nu) for t = 1:T-1] 
w = [zeros(nw) for t = 1:T]
x̄ = rollout(model, x1, ū, w)

# ## objective 
ot = (x, u, w) -> 0.1 * dot(x[3:4], x[3:4]) + 0.1 * dot(u, u)
oT = (x, u, w) -> 0.1 * dot(x[3:4], x[3:4])
ct = Cost(ot, nx, nu; nw=nw)
cT = Cost(oT, nx, 0; nw=nw)
obj = [[ct for t = 1:T-1]..., cT]

# ## constraints
goal(x, u, w) = x - xT

cont = Constraint()
conT = Constraint(goal, nx, 0)
cons = [[cont for t = 1:T-1]..., conT] 

# ## problem
prob = solver(model, obj, cons)
initialize_controls!(prob, ū)
initialize_states!(prob, x̄)

# ## solve
solve!(prob)

# ## solution
x_sol, u_sol = get_trajectory(prob)

# ## visuals
plot(hcat(x_sol...)')
plot(hcat(u_sol...)', linetype=:steppost)
